In [1]:
import numpy as np
from ase.build import molecule
from ase.visualize import view
import sys
sys.path.insert(0,"/home/valeria/Documents/DFTPY/PseudoP/vg/pyscf")
from pyscf.pbc.tools.pyscf_ase import atoms_from_ase
from pyscf import gto,dft
import matplotlib.pyplot as plt
from pyscf import scf

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from scipy.ndimage import convolve
from scipy import signal
from scipy.ndimage import gaussian_filter


#### Porpose of thi code
To model the effect of an external potential, we begin by generating the density matrix of a water molecule and 1000 external potentials. The application of each potential induces a change in the density matrix, which we express through the mapping:
$$
\delta \gamma \leftrightarrow \delta v
$$
This enables us to investigate the relationship between changes in the external potential and changes in the density matrix. We can learn this relationship using machine learning.

In [2]:
atoms = molecule('H2O')
basis = 'sto-3g'#'6-31g*'
xc = 'lda,vwn_rpa'
charge = 0

mol = gto.M(atom = atoms_from_ase(atoms), spin=0, basis=basis, charge = charge, parse_arg = False)

In [3]:
def random_v(shape,eps):
    bla=[np.random.random(shape) for _ in range(100)]
    bla=(bla+bla.T)/2.0
    return bla*eps

In [4]:
mf = scf.RKS(mol)
mf.verbose = 0
mf.run()

RKS object of <class 'pyscf.dft.rks.RKS'>

In [5]:
g_init = mf.make_rdm1
o=mf.mo_occ
c=mf.mo_coeff
g_new=np.einsum('i,mi,ni->mn',o,c,c)

In [6]:
kin = mol.intor('int1e_kin')
pot = mol.intor('int1e_nuc')

In [7]:
shape = np.shape(pot)
d_vext, d_gamma = [], []
for e in np.linspace(start=0.01, stop=0.1,num=10):
    perb, g_gamma = [], []
    for n in np.arange(0,100,1):
        d_pot = np.random.random(shape)
        randpot1 = (d_pot+d_pot.T)/2.0 * e
        randpot2 = randpot1 - np.trace(randpot1)* 1/len(g_new)
        randpot = gaussian_filter(randpot2,1)
        mf = scf.RKS(mol)
        mf.verbose = 0
        bla=mf.get_hcore
        def my_get_hcore(mol):
                hcore=bla(mol)
                hcore+=randpot
                return hcore
        mf.get_hcore = my_get_hcore
        mf.run(conv_tol=1.0e-12)
        g_perb = mf.make_rdm1()
        d_gama = g_new - g_perb
        g_gamma.append(d_gama)
        perb.append(randpot)
    d_vext.append(perb)
    d_gamma.append(g_gamma)

In [14]:
import pickle 
with open('/home/valeria/Documents/DFTPY/project_2/DATA/d_ext_smooth', 'wb') as f:
    pickle.dump(np.asarray(d_vext),f)
with open('/home/valeria/Documents/DFTPY/project_2/DATA/g_gamma_smooth', 'wb') as f:
    pickle.dump(np.asarray(d_gamma),f)

In [3]:
import pickle
with open('/home/valeria/Documents/DFTPY/project_2/DATA/d_ext_smooth', 'rb') as f:
    d_vext = pickle.load(f)
with open('/home/valeria/Documents/DFTPY/project_2/DATA/g_gamma_smooth', 'rb') as f:
    d_gamma = pickle.load(f)

In [4]:
D_vext = []
D_gamma =[]
for i in range(len(d_vext)):
    for j in range(len(d_vext[1])):
        D_vext.append(d_vext[i][j])
        D_gamma.append(d_gamma[i][j])

In [5]:
from sklearn.decomposition import PCA

v = np.asarray(D_vext)
vext = v.reshape(len(v), len(D_vext[1])*len(D_vext[1]))
pca = PCA(n_components = 0.95)

v_pca = pca.fit_transform(vext)

dist = np.zeros((1000, 1000))
for i in range(1000):
    for j in range(i+1, 1000):
        dist[i,j] = np.linalg.norm(v_pca[i,:] - v_pca[j,:])
        dist[j,i] = dist[i,j]

threshold = 0.01

similar_indices = np.where(dist < threshold)

# Remove duplicates
unique_indices = []
for i in range(len(similar_indices[0])):
    if similar_indices[0][i] < similar_indices[1][i]:
        unique_indices.append(similar_indices[0][i])

# Remove similar matrices from the dataset
unique_v = np.delete(v, unique_indices, axis=0)
unique_D_vext = np.delete(D_vext, unique_indices, axis=0)
unique_D_gamma = np.delete(D_gamma, unique_indices, axis=0)

In [7]:
import pickle 
with open('/home/valeria/Documents/DFTPY/project_2/DATA/d_vext_pca', 'wb') as f:
    pickle.dump(unique_v,f)
with open('/home/valeria/Documents/DFTPY/project_2/DATA/g_gamma_pca', 'wb') as f:
    pickle.dump(unique_D_gamma,f)